<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## ***DATA SCIENCE / SECTION 4 / SPRINT 2 / NOTE 4***

---

# Transformer & BERT, GPT

## 🏆 학습 목표

- **Transformer의 장점과 주요 프로세스인 Self-Attention에 대해 이해하고 설명할 수 있다.** 
    - 트랜스포머를 발표한 논문 제목은 왜 "Attention is All You Need"인지 설명할 수 있다.
    - Positional Encoding을 적용하는 이유에 대해서 설명할 수 있다.
    - Masked Self-Attention가 트랜스포머 구조 중 어디에 적용되며 어떤 역할을 하는지 설명할 수 있다. 
    - 기존 RNN과 비교하여 Transformer가 가지는 장점에 대해서 설명할 수 있다.

- **GPT, BERT 그리고 다른 모델에 대해서 개략적으로 설명할 수 있다.**
    - GPT(Generative Pre-Training)
        - 사전 학습된 언어 모델(Pre-trained LM)의 Pre-training과 Fine-tuning은 무엇이고 각각 어떤 종류의 데이터셋을 사용하는 지 설명할 수 있다.
        - GPT는 Transformer를 어떻게 변형하였는지 설명할 수 있다.
    - BERT(Bidirectional Encoder Representation by Transformer)
        - BERT는 Transformer를 어떻게 변형하였으며 GPT와의 차이 무엇인지 알 수 있다.
        - MLM(Masked Language Model)은 무엇인지 이해할 수 있다.
        - NSP(Next Sentence Prediction)은 무엇인지 이해할 수 있다.
- **최근 언어 모델의 발전은 어떻게 진행되고 있는지 알 수 있다.**

- 주의 : 이후에도 새로운 모델을 공부하게 되면 아래와 같은 순서로 학습하게 될 겁니다.
    1. 논문을 통해 해당 모델의 컨셉과 대략적인 구조를 파악한다.
    2. 기존에 구현되어 있는 라이브러리를 통해 해당 모델을 사용해본다. (추론 혹은 파인튜닝)
    3. 해당 모델에 대한 좀 더 깊은 이해가 필요할 경우 코드로 구현해본다.
    - 즉, 오늘은 첫 번째로 아래 개념을 배우는 만큼 모든 코드를 이해하지 못해도 괜찮습니다.


## Warm Up

지난 시간에 배웠던 내용을 떠올려봅시다.

- RNN, LSTM, GRU

    - RNN 기반 모델의 장점에 대해서 생각해봅시다.
    - RNN 기반 모델의 단점에 대해서 생각해봅시다. (2가지 이상)
        - LSTM과 GRU는 어떤 단점을 어떻게 극복하였는지 다시 알아봅시다.

- 이번 시간에는 Transformer에 대해서 배울 예정입니다.
    - [Transformer](https://www.youtube.com/watch?v=mxGCEWOxfe8) 소개 영상
        - RNN과는 완전히 다른 모델
        - ***Attention is All You Need : 필요한 건 Attention 뿐***
    - [GPT](https://www.youtube.com/watch?v=FeEmmylAF0o) 소개 영상
        - 사전 학습 언어 모델
        - GPT의 구조
    - [BERT](https://youtu.be/vo3cyr_8eDQ?t=712) 소개 영상 (11:52 부터)
        - BERT의 구조
        - MLM(Masked Self-Attention)
        - NSP(Next Sentence Prediction)

## 1. Transformer : Attention is All You Need

### 트랜스포머(Transformer)란?

<img src="https://upload.wikimedia.org/wikipedia/en/thumb/9/94/Transformers_Age_of_Extinction_Poster.jpeg/220px-Transformers_Age_of_Extinction_Poster.jpeg" alt="transformer_electric" width="200"/>

트랜스포머는 기계 번역을 위한 새로운 모델로 이전에 등장했던 Attention 메커니즘을 극대화하여 뛰어난 번역 성능을 기록했습니다.<br/>
최근 자연어처리 모델 SOTA(State-of-the-Art)의 기본 아이디어는 거의 모두 트랜스포머를 기반으로 하고 있습니다.<br/>
모델을 소개한 논문 [Attention is All You Need](https://arxiv.org/abs/1706.03762) 는 3년 사이에 18000번 이상 인용되었습니다.<br/>
트랜스포머가 자연어처리가 아닌 다른 문제도 잘 풀고있기 때문에 최근에는 컴퓨터 비전 쪽에서도 적용하려는 시도가 있으며, 멀티모달(Multi-Modal) 모델에도 적용되고 있습니다.<br/>

RNN 기반 모델이 가진 구조적 단점은 단어가 **순서대로** 들어온다는 점입니다.<br/>
그렇기 때문에 처리해야 하는 시퀀스가 길수록 **연산 시간이** 길어집니다.<br/>
**트랜스포머는 이런 문제를 해결하기 위해 등장한 모델**입니다.<br/>
모든 토큰을 동시에 입력받아 병렬 연산하기 때문에 GPU 연산에 최적화 되어 있습니다.

아래는 트랜스포머의 구조를 단순하게 시각화한 그림입니다.<br/>
Encoder, Decoder로 표현된 사각형을 각각 인코더 블록과 디코더 블록이라고 합니다.<br/>
트랜스포머는 인코더 블록과 디코더 블록이 6개씩 모여있는 구조를 하고 있습니다. 

<img src="http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png" alt="positional_encoding" width="700" />

그림 하나를 더 보도록 하겠습니다. 아래는 논문에서 트랜스포머의 구조를 나타낸 그림입니다.<br/>
그림을 보면 커다란 회색 블록이 2개 있습니다.<br/>
왼쪽은 인코더 블록 하나를 나타내고 오른쪽은 디코더 블록 하나를 나타냅니다.<br/>
인코더 블록은 크게 2개의 sub-layer [**`Multi-Head (Self) Attention`, `Feed Forward`**] 로 나눌 수 있습니다.<br/>
디코더 블록은 3개의 sub-layer [**`Masked Multi-Head (Self) Attention`, `Multi-Head (Encoder-Decoder) Attention`, `Feed Forward`**] 로 나눌 수 있습니다.

<img src="https://miro.medium.com/max/1400/1*BHzGVskWGS_3jEcYYi6miQ.png" alt="positional_encoding" width="550" />

### Positional Encoding (위치 인코딩)

<img width="300" alt="pe" src="https://user-images.githubusercontent.com/45377884/112799904-ecb3a100-90a9-11eb-9072-87a965e81a77.png">

트랜스포머에서는 병렬화를 위해 모든 단어 벡터를 동시에 입력받습니다.<br/>
컴퓨터는 어떤 단어가 어디에 위치하는지 알 수 없게 됩니다.<br/>
그래서 컴퓨터가 이해할 수 있도록 단어의 위치 정보를 제공하기 위한 벡터를 따로 제공해주어야 합니다.<br/>
단어의 상대적인 위치 정보를 제공하기 위한 벡터를 만드는 과정을 `Positional Encoding` 이라고 합니다.<br/>

`Positional Encoding`은 아래와 같은 수식으로 이루어집니다.<br/>
수식이 복잡하니 수학적으로 이해하려고 하지 않으셔도 됩니다.

$$
\begin{aligned}
\text{PE}_{\text{pos},2i} &= \sin \bigg(\frac{\text{pos}}{10000^{2i/d_{\text{model}}}}\bigg) \\
\text{PE}_{\text{pos},2i+1} &= \cos \bigg(\frac{\text{pos}}{10000^{2i/d_{\text{model}}}}\bigg)
\end{aligned}
$$


아래는 `Positional Encoding`을 통해 만들어진 벡터를 시각화한 자료입니다.<br/>
일정한 패턴이 있는 벡터가 만들어지는 것을 볼 수 있습니다.<br/>
컴퓨터는 이를 통해 단어의 상대적인 위치를 파악하게 됩니다.<br/>

<img src="http://jalammar.github.io/images/t/transformer_positional_encoding_large_example.png" alt="positional_encoding" width="500" />

In [ ]:
def get_angles(pos, i, d_model):
    """
    sin, cos 안에 들어갈 수치를 구하는 함수입니다.
    """
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

In [ ]:
def positional_encoding(position, d_model):
    """
    위치 인코딩(Positional Encoding)을 구하는 함수입니다.
    
    """
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

### Self-Attention (셀프-어텐션)
***(이번 노트에서는 이것만이라도 알고 넘어가보도록 합시다)***






<img width="300" alt="self-Attn" src="https://user-images.githubusercontent.com/45377884/112809266-ca735080-90b4-11eb-9a25-7f34f37880c7.png">

트랜스포머의 주요 메커니즘인 **Self-Attention**에 대해 알아보도록 하겠습니다.

아래와 같은 문장이 있다고 해보겠습니다.

> *The animal didn't cross the street because <ins>it</ins> was too tired* 

위 문장을 제대로 번역하려면 **_"it"_** 과 같은 지시대명사가 어떤 대상을 가리키는지 알아야 합니다.<br/>
그렇기 때문에 트랜스포머에서는 번역하려는 문장 내부 요소의 관계를 잘 파악하기 위해서, 문장 자신에 대해 어텐션 메커니즘을 적용합니다.<br/>
이를 `Self-Attention`이라고 합니다.

아래는 **_"it"_** 이 어떤 단어와 가장 연관되어 있는 지를 시각화한 그림입니다.

<img src="http://jalammar.github.io/images/t/transformer_self-attention_visualization.png" alt="self_attention_visualization" width="350" />

**`Self-Attention`**은 어떤 과정이길래 단어 사이의 관계를 알아낼 수 있을까요?

Self-Attention에서도 쿼리(Query)-키(Key)-밸류(Value)의 아이디어가 동일하게 등장합니다.


----
- **지난 시간에 배운 내용을 복습해봅시다 - 검색 시스템에서 흘러나온 쿼리, 키, 밸류**

Attention을 다룰 때 등장했던 검색 시스템의 아이디어를 다시 복습해봅시다.

아래는 구글에서 _"what is attention in nlp"_ 라는 검색어를 구글에 입력했을 때의 검색 결과를 나타낸 이미지입니다.

<img src="https://i.imgur.com/JdCQr1l.png" alt="retrieval_system" width="600" />

그림에서 볼 수 있듯이 검색 시스템은 아래와 같은 3단계를 거쳐 작동합니다.

1. 찾고자 하는 정보에 대한 검색어(Query)를 입력합니다.
2. 검색 엔진은 검색어와 가장 비슷한 키워드(Key)를 찾습니다.
3. 그리고 해당 키워드(Key)와 연결된 페이지(Value)를 보여줍니다.
-------

기존 Attention과의 차이는 각 벡터가 모두 가중치 벡터라는 점입니다.<br/>

각각의 벡터가 어떤 역할을 하는지 알아보겠습니다.

- **쿼리(q)**는 분석하고자 하는 단어에 대한 가중치 벡터입니다.

- **키(k)**는 각 단어가 쿼리에 해당하는 단어와 얼마나 연관있는 지를 비교하기 위한 가중치 벡터입니다.

- **밸류(v)**는 각 단어의 의미를 살려주기 위한 가중치 벡터입니다.

**`Self-Attention`**은 세 가지 가중치 벡터를 대상으로 어텐션을 적용합니다.<br/>
적용하는 방식은 기존 Attention 메커니즘과 거의 동일합니다.

1. 먼저, **특정 단어의 쿼리(q) 벡터와 모든 단어의 키(k) 벡터를 내적**합니다. 내적을 통해 나오는 값이 Attention 스코어(Score)가 됩니다.

2. 트랜스포머에서는 이 가중치를 q,k,v 벡터 차원 $d_k$ 의 제곱근인 $\sqrt{d_k}$ 로 나누어줍니다.<br/>계산값을 안정적으로 만들어주기 위한 계산 보정으로 생각해주시면 됩니다.  

3. 다음으로 **Softmax**를 취해줍니다.<br/>
이를 통해 쿼리에 해당하는 단어와 문장 내 다른 단어가 가지는 관계의 비율을 구할 수 있습니다.

4. 마지막으로 **밸류(v) 각 단어의 벡터를 곱해준 후 모두 더하면** Self-Attention 과정이 마무리됩니다. 

**`Self-Attention`** 의 과정을 그림으로 다시 보겠습니다.

**1. 가중치 행렬 $W^Q, W^K, W^V$ 로부터 각 단어의 쿼리, 키, 밸류(q, k, v) 벡터를 만들어냅니다.**

<img src="http://jalammar.github.io/images/xlnet/self-attention-1.png" alt="transformer_15" width="600" />

**2. 분석하고자 하는 단어의 쿼리 벡터(q)와 문장 내 모든 단어(자신 포함)의 키 벡터(k)를 내적하여 각 단어와 얼마나 관련 정도를 구합니다.**

(아래 그림에서는 $\sqrt{d_k}$로 나누어 준 뒤에 Softmax를 취해주는 과정은 생략되었습니다.)


<img src="http://jalammar.github.io/images/xlnet/self-attention-2.png" alt="transformer_15" width="600" />

**3.  Softmax의 출력값과밸류 벡터(v)를 곱해준 뒤 더하면 해당 단어에 대한 Self-Attention 출력값을 얻을 수 있습니다.**

<img src="http://jalammar.github.io/images/xlnet/self-attention-3.png" alt="transformer_15" width="600" />

**4. 하나의 벡터에 대해서만 살펴보았지만 실제 Attention 계산은 행렬 단위로 병렬 계산됩니다.**

<img src="http://jalammar.github.io/images/xlnet/self-attention-summary.png" alt="transformer_15" width="600" />

실제로 각 벡터는 **행렬(Q, K, V)**로 한꺼번에 계산됩니다. $W^Q, W^K, W^V$ 는 학습 과정에서 갱신되는 파라미터로 이루어진 행렬입니다.<br/>
세 행렬과 단어 행렬을 내적하여 쿼리, 키, 밸류 행렬(Q, K, V)를 만들어냅니다.



<img src="http://jalammar.github.io/images/t/self-attention-matrix-calculation.png" alt="transformer_12" width="500" />

위에서 살펴본 바와 같이

1. 먼저 쿼리 행렬(Q)과 키 행렬(K)을 **내적**합니다.

2. 결과로 나오는 행렬의 요소를 $\sqrt{d_k}$ 로 **나누어 줍니다.**

3. 행렬의 각 요소에 `Softmax`를 취해줍니다. 

4. 마지막으로 **밸류 행렬(V)과 내적**하면 최종 결과 행렬(Z)이 반환됩니다.

<img src="http://jalammar.github.io/images/t/self-attention-matrix-calculation-2.png" alt="transformer_13" width="700" />

아래는 `Tensorflow` 에서 Self-Attention을 구현한 코드입니다. 코드를 통해 Attention이 계산되는 과정을 다시 살펴보도록 합시다. 

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

### Multi-Head Attention

다음으로 **`Multi-Head Attention`** 에 대해 알아보겠습니다.<br/>
**`Multi-Head Attention`** 이란 **`Self-Attention`** 을 동시에 여러 개로 실행하는 것입니다.<br/>
각 Head 마다 다른 Attention 결과를 내어주기 때문에 앙상블과 유사한 효과를 얻을 수 있습니다.<br/> 
논문에서는 8개의 Head를 사용하였습니다.<br/>
8번의 Self-Attention을 실행하여 각각의 출력 행렬 $Z_0, Z_1, \cdots , Z_7$ 을 만들어냅니다.

<img src="http://jalammar.github.io/images/t/transformer_attention_heads_z.png" alt="transformer_16" width="500"/>

출력된 행렬 $Z_n (n=0,\cdots,7)$ 은 **이어붙여집니다(Concatenate)**.<br/>
또 다른 파라미터 행렬인 $W^o$ 와의 내적을 통해 Multi-Head Attention의 최종 결과인 행렬 $Z$를 만들어냅니다.<br/>
여기서 행렬 $W^o$의 요소 역시 학습을 통해 갱신됩니다.<br/>
최종적으로 생성된 행렬 $Z$는 토큰 벡터로 이루어진 행렬 $X$와 **동일한 크기(Shape)**가 됩니다.

<img src="http://jalammar.github.io/images/t/transformer_attention_heads_weight_matrix_o.png" alt="transformer_17" width="500" />

### Layer Normalization & Skip Connection



<img width="300" alt="lnorm_resicon" src="https://user-images.githubusercontent.com/45377884/113169444-9056aa00-9280-11eb-8ba0-17c9211ad412.png">

트랜스포머의 모든 sub-layer에서 출력된 벡터는 **Layer normalization**과 **Skip connection**을 거치게 됩니다.<br/>
Layer normalization의 효과는 Batch normalization과 유사합니다. 학습이 훨씬 빠르고 잘 되도록 합니다.<br/>
Skip connection(혹은 Residual connection)은 역전파 과정에서 정보가 소실되지 않도록 합니다.<br/>
Sprint 3에서 배울 ResNet의 주요 메커니즘이므로 해당 부분에서 더욱 자세하게 다룰 예정입니다.

### Feed Forward Neural Network

<img width="300" alt="스크린샷 2021-03-29 오후 5 27 32" src="https://user-images.githubusercontent.com/45377884/112808809-58027080-90b4-11eb-8ca7-ffa38e577d3d.png">



다음으로 **`FFNN(Feed forward neural network)`** 로 들어갑니다.<br/>은닉층의 차원이 늘어났다가 다시 원래 차원으로 줄어드는 단순한 2층 신경망입니다.<br/>활성화 함수(Activation function)으로 ReLU를 사용합니다.

$$
 \text{FFNN}(x) = \max(0, W_1x + b_1) W_2 +b_2
$$

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

### Masked Self-Attention


<img width="300" alt="Masked_Self-Attention_in_structure" src="https://user-images.githubusercontent.com/45377884/112808936-78322f80-90b4-11eb-9315-22cd9caad41d.png">

**Masked Self-Attention**은 디코더 블록에서 사용되는 특수한 Self-Attention입니다.<br/>
디코더는 Auto Regressive 하게 단어를 생성하기 때문에 타깃 단어 이후 단어를 보지 않고 단어를 예측해야 합니다.<br/>
따라서 타깃 단어 뒤에 위치한 단어는 Self-Attention에 영향을 주지 않도록 **마스킹(masking)**을 해주어야 합니다.

<img width="500" alt="Masked_Self-Attention_ex" src="http://jalammar.github.io/images/xlnet/transformer-decoder-block-self-attention-2.png">

***Self-Attention (without Masking) vs Masked Self-Attention***

<img width="500" alt="Masked_Self-Attention_ex2" src="http://jalammar.github.io/images/gpt2/self-attention-and-masked-self-attention.png">

Self-Attention 메커니즘은 쿼리 행렬(Q)와 키 행렬(K)의 내적합니다.<br/>
결과로 나온 행렬을 차원의 제곱근 $\sqrt{d_k}$ 로 나누어 준 다음,<br/> 
Softmax를 취해주고 밸류 행렬(V)과 내적하였습니다.

**`Masked Self-Attention`** 에서는 Softmax를 취해주기 전, 가려주고자 하는 요소에만 $-\infty$ 에 해당하는 매우 작은 수를 더해줍니다.<br/>
아래 예시에서는 -10억(=-1e9)을 더해주었습니다.<br/>
이 과정을 **마스킹(Masking)**이라고 합니다.<br/>
마스킹된 값은 Softmax를 취해 주었을 때 0이 나오므로 Value 계산에 반영되지 않습니다.

<img width="600" alt="masked_1" src="http://jalammar.github.io/images/gpt2/transformer-attention-mask.png">

<img width="600" alt="masked_2" src="http://jalammar.github.io/images/gpt2/transformer-attention-masked-scores-softmax.png">

위에서 등장했던 Self-Attention을 구현 코드에서. `mask` 와 관련된 부분만 다시 보도록 합시다.

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead) 
  but it must be broadcastable for addition.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    
  """
    mask가 있을 경우 masking된 자리(mask=1)에는 (-inf)에 해당하는 절댓값이 큰 음수 -1e9(=-10억)을 더해줍니다.
    그 값에 softmax를 취해주면 거의 0에 가까운 값이 나옵니다. 그 다음 value 계산시에 반영되지 않습니다.
  """
    
  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  


  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

### Encoder-Decoder Attention

<img width="300" alt="Encoder-Decoder_Attention" src="https://user-images.githubusercontent.com/45377884/112809435-f8f12b80-90b4-11eb-96e1-3b0f7c530659.png">

디코더에서 Masked Self-Attention 층을 지난 벡터는 **Encoder-Decoder Attention** 층으로 들어갑니다.<br/>
좋은 번역을 위해서는 번역할 문장과 번역된 문장 간의 관계 역시 중요합니다.<br/>
번역할 문장과 번역되는 문장의 정보 관계를 엮어주는 부분이 바로 이 부분입니다.

이 층에서는 **디코더 블록의** Masked Self-Attention으로부터 출력된 벡터를 **쿼리(Q)** 벡터로 사용합니다.<br/>
**키(K)와 밸류(V)** 벡터는 최상위(=6번째) 인코더 블록에서 사용했던 값을 그대로 가져와서 사용합니다.<br/>
**`Encoder-Decoder Attention`** 층의 계산 과정은 Self-Attention 했던 것과 동일합니다.

아래는 **`Encoder-Decoder Attention`** 가 진행되는 순서를 나타낸 이미지입니다.

<img width="700" alt="Encoder-Decoder_Attention_gif" src="http://jalammar.github.io/images/t/transformer_decoding_1.gif">

### Linear & Softmax Layer

이제 끝입니다!

<img width="300" alt="Linear_Softmax" src="https://user-images.githubusercontent.com/45377884/112815762-994a4e80-90bb-11eb-9a57-a8be65c1a30b.png">

디코더의 최상층을 통과한 벡터들은 Linear 층을 지난 후 Softmax를 통해 예측할 단어의 확률을 구하게 됩니다.

### 코드 실습 1

(오래 걸립니다. 원활한 실습을 위해서 epoch 수를 줄이고 학습해도 좋습니다.)

필요한 모듈을 import 해줍니다.

In [ ]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers.experimental.preprocessing import TextVectorization

학습팔 말뭉치를 불러옵니다.<br/>
아래 코드에서는 스페인어 - 영어 말뭉치를 불러옵니다.

In [ ]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

2646016/2638744 [==============================] - 0s 0us/step


학습할 수 있도록 적절히 전처리를 해줍니다.

In [ ]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

전처리가 잘 되었는지 확인합니다. <br/>
번역되는 문장의 앞에는 [start] 토큰을 위치시키고 뒤에는 [end] 토큰을 위치시킵니다.

In [ ]:
for _ in range(5):
    print(random.choice(text_pairs))

('You deserve a medal.', '[start] Te mereces una medalla. [end]')
("Tom doesn't owe me anything.", '[start] Tom no me debe nada. [end]')
('I asked the boy to throw the ball back.', '[start] Le pedí al niño que me devolviera la pelota. [end]')
('I drank from the tap.', '[start] He bebido del grifo. [end]')
('Tom paid way too much for that old car.', '[start] Tom pagó demasiado por este carro viejo. [end]')


데이터셋을 split 해줍니다.

In [ ]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


In [ ]:
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


eng_vectorization = TextVectorization(
    max_tokens=vocab_size, output_mode="int", output_sequence_length=sequence_length,
)
spa_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

In [ ]:

def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return ({"encoder_inputs": eng, "decoder_inputs": spa[:, :-1],}, spa[:, 1:])


def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)


In [ ]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [ ]:
epochs = 30  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
positional_embedding (Positiona (None, None, 256)    3845120     encoder_inputs[0][0]             
__________________________________________________________________________________________________
decoder_inputs (InputLayer)     [(None, None)]       0                                            
__________________________________________________________________________________________________
transformer_encoder (Transforme (None, None, 256)    3155456     positional_embedding[0][0]       
________________________________________________________________________________________

In [ ]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(30):
    input_sentence = random.choice(test_eng_texts)
    translated = decode_sequence(input_sentence)

### 코드 실습 2

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val)
)

## 2.GPT, BERT

다음으로 GPT와 BERT에 대해서 알아보겠습니다.<br/>

GPT와 BERT는 트랜스포머 구조를 변형하여 만들어진 언어 모델입니다.<br/>
두 모델은 **사전 학습된 언어 모델(Pre-trained Language Model)** 이라는 공통점을 가지고 있습니다.<br/>
사전 학습이란 대량의 데이터를 사용하여 미리 학습하는 과정입니다. 여기에 필요한 데이터를 추가 학습시켜 모델의 성능을 최적화합니다.<br/>
이런 학습 방법을 전이 학습(Transfer Learning)이라고도 합니다.<br/>
최근 발표되고 있는 언어 모델은 모두 전이 학습을 적용하고 있습니다.

### GPT (2018.6)

<img width="300" alt="Linear_Softmax" src="https://openai.com/content/images/2019/05/openai-cover.png">


**GPT**는 **G**enerative **P**re-trained **T**ransformer 의 약자로 2018년 6월에 OpenAI를 통해 발표되었습니다.<br/>
연이어 발표한 GPT-2(2019.2), GPT-3(2020.6)가 좋은 성능을 보이면서 세간의 주목을 받았습니다.<br/>
아래는 관련 기사이니 세션이 끝나고 추가적으로 읽어보시면 좋겠습니다. 

- 심화내용 : Generative란? (당장은 이해하지 못하셔도 좋습니다) - [discriminative vs generative](https://ratsgo.github.io/generative%20model/2017/12/17/compare/)

- GPT2 기사
    - [The AI that was too dangerous to release](https://blog.floydhub.com/gpt2/)
    - [OpenAI, 공유하기에는 너무 위험한 ‘텍스트 생성 AI’의 진실](http://www.aitimes.com/news/articleView.html?idxno=121589)

- GPT3 기사
    - [A GPT-3 bot posted comments on Reddit for a week and no one noticed](https://www.technologyreview.com/2020/10/08/1009845/a-gpt-3-bot-posted-comments-on-reddit-for-a-week-and-no-one-noticed/)
    - [GPT3가 쓴 뉴스가 랭킹 1위, 사람을 이겼다](http://www.aitimes.com/news/articleView.html?idxno=131593)



GPT-1, GPT-2, GPT-3 의 구조가 전부 동일하지는 않지만 기본적인 뼈대는 동일합니다.

GPT의 구조를 본격적으로 알아보기에 앞서 기본이 되는 아이디어인 **사전 학습(Pre-training)**에 대해서 알아보겠습니다.

- **사전 학습된 언어 모델 (Pre-trained LM)**

혹시 아래와 같은 케이스를 본 적 있으신가요?

<img width="500" alt="pre-train" src="https://user-images.githubusercontent.com/45377884/112774307-3e413900-9074-11eb-94ab-f3bc000ff95e.png">


수능 국어 시험에서 책을 많이 읽은 학생은 조금만 공부해도 상위권 성적을 유지하는 경우가 종종 있습니다.<br/> GPT에서 사용된 **사전 학습**이라는 아이디어도 유사한 생각에서 시작되었습니다.<br/>아래 그림을 보며 사전 학습 언어 모델에 대해서 설명을 이어나가겠습니다.

<img width="700" alt="Pre-training" src="https://user-images.githubusercontent.com/45377884/112943247-35cc2980-916c-11eb-99be-2fa7657507d2.png">

사전 학습 언어 모델은 크게 2가지 과정을 통해 완성됩니다. 첫 번째가 **사전 학습(Pre-training)**입니다. 

존재하는 자연어 중에는 책(Book corpus)이나 위키피디아(Wiki corpus) 등 레이블링 되지 않은 데이터가 더 많습니다.<br/>
여러분이 지금 읽고 있는 강의 노트 역시 레이블링 되지 않은 자연어 데이터 입니다.<br/>
책을 많이 읽는 것처럼 레이블링 되지 않은 데이터를 모델이 학습하도록 하는 과정을 **사전 학습** 이라고 합니다.

사전 학습이 끝난 모델에 우리가 하고자하는 태스크에 특화된(Task specific) 데이터를 학습합니다.<br/>
이를 **Fine-tuning** 이라고 합니다.<br/> Fine-tuning에서는 학습시 레이블링 된 데이터 [Ex) 감성 분석, 자연어 추론(NLI), 질의 응답(QA)] 를 사용합니다.

- **모델 구조 (그림은 6개의 디코더 블록을 사용하지만 GPT는 12개의 블록을 사용합니다.)**

<img width="700" alt="Pre-training" src="http://jalammar.github.io/images/xlnet/transformer-decoder-intro.png">

GPT에서는 인코더를 사용하지 않기 때문에 디코더 블록내에 **2개의 Sub-layer**만 있습니다.<br/>
트랜스포머의 디코더 블록에는 Masked Self-Attention, Encoder-Decoder Attention, Feed-Forward 층이 있었습니다.<br/>
하지만 GPT는 인코더를 사용하지 않기 때문에 Encoder-Decoder Attention 층이 빠지게 됩니다.

- **사전 학습(Pre-training)**

레이블링 되지 않은 대량의 말뭉치 $U = (u_1, \cdots , u_n)$ 에 대해 로그 우도 $L_1$ 을 최대화하는 방향으로 학습됩니다.<br/>
지난 노트에서 배웠던 언어 모델처럼 다음에 올 단어를 계속해서 맞추는 방식으로 학습합니다.

$$
L_1(U) = \sum_i \log P(u_i \vert u_{i-k}, \cdots, u_{i-1}; \Theta)
$$

- **Fine-tuning**

기존 모델에서는 태스크에 맞춰 모델 구조를 변경해주고 학습을 진행시켰습니다.

하지만 GPT와 같은 사전 학습 언어 모델은 Fine-tuning 과정에서 데이터의 입력 방식만을 변형시키고 모델 구조는 일정하도록 설계되었습니다.

<img width="600" alt="fine-tune_structure" src="https://user-images.githubusercontent.com/45377884/112949500-408abc80-9174-11eb-8090-4f0be34db572.png">


Fine-tuning은 레이블링 된 말뭉치 $C = (x_1, \cdots , x_m)$ 에 대하여 로그 우도 $L_2$ 를 최대화하는 방향으로 학습합니다.

$$
L_2(C) = \sum_{(x,y)} \log P(y \vert x_1, \cdots , x_m)
$$

GPT의 경우 Fine-tuning에서 학습하는 데이터셋이 클 때는 보조 목적함수로 $L_1$ 을 추가하여 $L_3$로 학습하면 학습이 더 잘 진행됩니다.

$$
L_3(C) = L_2(C) + \lambda \cdot L_1(C)
$$

- **결과 & 결론**

LSTM, GRU를 사용한 기존 모델보다 자연어 추론(NLI), 질의 응답(QA), 분류(Classification) 등의 태스크에서 높은 성능을 달성하였습니다.<br/>GPT는 사전 학습된 언어 모델을 바탕으로 좋은 성능을 확보할 수 있다는 점과 사전 학습 모델에 Transformer 구조가 더 좋은 성능을 보임을 알 수 있었습니다.

### BERT (2018.10)

<img width="600" alt="model_name" src="https://user-images.githubusercontent.com/45377884/112963631-88184500-9182-11eb-8c87-f470e25d7567.png">

> [다양한 캐릭터 이름을 딴 NLP 모델](https://towardsdatascience.com/machine-learnings-obsession-with-kids-tv-show-characters-728edfb43b3c) : 재미(?)로 읽어보세요 :)

**BERT**(**B**idirectional **E**ncoder **R**epresentation by **T**ransformer)는 2018년 10월 구글에서 발표한 모델입니다.

모델 이름에서 알 수 있듯 BERT는 트랜스포머의 인코더만을 사용하여 문맥을 양방향(Bidirectional)으로 읽어냅니다.

- **BERT의 구조**






GPT가 트랜스포머의 디코더 블록을 12개 쌓아올린 모델이었다면 **BERT는 트랜스포머의 인코더 블록**을 12개 쌓아올린 모델입니다.


<img width="500" alt="model_name" src="http://jalammar.github.io/images/bert-base-bert-large-encoders.png">

- **BERT의 Special Token ([CLS], [SEP])과 입력 벡터를 알아봅시다**

이번에는 BERT에만 있는 특별한 토큰과 이에 따라 입력되는 벡터가 어떻게 구성되는 지에 대해 알아보겠습니다.

<img width="600" alt="bert_input" src="https://imgur.com/iW77E5Q.png">

1. Special Token : **`[CLS]`**, **`[SEP]`**



BERT에는 **`[CLS]`**와 **`[SEP]`**이라는 두 가지 Special Token이 있습니다.<br/>

> **`[CLS]`** : Classification <br/>

[CLS] 토큰은 입력의 맨 앞에 위치하는 토큰입니다.<br/>
아래에서 등장할 NSP(Next Sentence Prediction)이라는 학습을 위해 존재합니다.

> **`[SEP]`** : Separation <br/>

BERT는 사전 학습 시에 텍스트를 2부분으로 나누어 넣게 됩니다.<br/>
**`[SEP]`** 토큰은 첫 번째 부분의 끝자리와 두 번째 부분의 끝자리에 위치합니다.

<img width="600" alt="bert_input" src="https://imgur.com/iW77E5Q.png">

2. Input Vector : Token Embeddings, Segment Embeddings, Position Embeddings

BERT는 3종류의 임베딩 벡터를 모두 더하여 모델에 입력합니다.

- Token Embeddings : 단어를 나타내는 임베딩입니다. Word2Vec, GloVe, FastText 등으로 사전 학습된 임베딩 벡터를 사용합니다.<br/>
- Segment Embeddings : 첫 번째 부분과 두 번째 부분을 구분하기 위한 임베딩입니다.<br/>**`[CLS]`** 부터 첫 번째 **`[SEP]`** 까지 동일한 벡터를 적용하고, 다음 토큰부터 두 번째 **`[SEP]`** 까지 동일한 벡터를 적용합니다.
- Positional Embeddings : 단어의 위치를 나타내기 위한 임베딩입니다.


- **BERT의 사전 학습(Pre-training) 방법들**


BERT 역시 Pre-trained Language Model이기 때문에 사전 학습 이후에 Fine-tuning을 진행하게 됩니다.<br/>
아래에서는 BERT의 사전 학습 방식에 대해 알아보겠습니다.<br/>
BERT는 GPT와 다른 방식의 사전 학습이 적용되었습니다.<br/>
BERT의 사전 학습에서 사용되는 **2가지 방법(MLM, NSP)**을 알아보겠습니다.

> **MLM(Masked Language Model)**

첫 번째는 MLM(Masked Language Model) 입니다. 혹시 아래와 같은 문제를 풀어보신 경험이 있으신가요?

<img width="300" alt="mlm" src="https://thumb.mt.co.kr/06/2013/11/2013110718224659109_1.jpg/dims/optimize/">

영어 시험을 한 번쯤 준비해보신 분이라면 빈칸 채우기 유형 문제를 풀어보신 적이 있을 것입니다.<br/>
위와 같은 문제에서 보통은 빈칸에 문법적/의미적으로 올바른 단어를 채우게 됩니다.

BERT도 이처럼 **빈칸 채우기**를 하면서 단어를 학습합니다.<br/>
BERT는 사전 학습 과정에서 레이블링 되지 않은 말뭉치 중에서 랜덤으로 15\%가량의 단어를 마스킹합니다.<br/>
그리고 마스킹된 위치에 원래 있던 단어를 예측하는 방식으로 학습을 진행합니다.

<img width="600" alt="mlm_example" src="http://jalammar.github.io/images/BERT-language-modeling-masked-lm.png">

MLM은 양쪽의 문맥을 동시에 볼 수 있다는 장점이 있습니다.

아래 그림은 GPT와 BERT의 학습 방향을 비교하여 나타낸 그림입니다.

<img width="300" alt="gpt_vs_bert" src="https://user-images.githubusercontent.com/45377884/113259927-a445ee80-9308-11eb-8fbd-95d5f553480a.png">

GPT는 ***'거기'*** 라는 단어를 예측할 때 '어제 카페 갔었어'의 정보만 볼 수 있습니다.<br/>
하지만 BERT는 빈칸에 들어갈 ***'거기'*** 라는 단어를 예측 할 때 '어제 카페 갔었어'뿐만 아니라 '사람 많더라'의 정보도 참고할 수 있습니다.<br/>
이렇게 양방향으로 학습할 경우 단어가 문맥 사이에서 가진 의미를 최대로 학습할 수 있습니다.

MLM은 다소 간단한 아이디어이지만 단어의 문맥적 의미를 최대로 학습할 수 있도록 함으로써 BERT가 높은 성능을 달성하는 데에 커다란 역할을 하였습니다.<br/>다음으로 2번째 방법인 NSP에 대해서 알아보겠습니다.

> **NSP(Next Sentence Prediction)**

BERT는 NSP(Next Sentence Prediction) 방식으로도 학습합니다. 동문서답이라는 사자성어 알고 계신가요?

<img width="350" alt="nsp_idea" src="https://thx.sfo2.cdn.digitaloceanspaces.com/wr/hanja_images/%E6%9D%B1%E5%95%8F%E8%A5%BF%E7%AD%94_800.jpg">

NSP는 모델이 문맥에 맞는 이야기를 하는지 아니면 동문서답을 하는지를 판단하며 학습하는 방식입니다.<br/>

위에서 알아본 **`[SEP]`** 토큰 왼쪽의 문장과 오른쪽의 문장이 바로 이어지는 문장일 경우 **`[CLS]`** 토큰의 출력이 **`IsNext`** 로 되도록 학습합니다.<br/> 
두 문장이 이어지지 않는 쌍일 경우에는 출력 벡터가 **`NotNext`** 로 나오도록 학습합니다.

<img width="500" alt="nsp_1" src="http://jalammar.github.io/images/bert-next-sentence-prediction.png">

아래는 드라마 대본을 예시로 NSP가 어떻게 작동하는 지를 나타낸 그림입니다.

<img width="500" alt="nsp_2" src="https://user-images.githubusercontent.com/45377884/86514846-d0067780-be4f-11ea-9809-c3e43b8ad3f9.png">     

<img width="500" alt="nsp_3" src="https://user-images.githubusercontent.com/45377884/86514847-d137a480-be4f-11ea-82be-d229bf75fbf8.png">

NSP 역시 간단한 아이디어입니다.<br/>
모델이 문장과 문장 사이의 관계를 학습할 수 있도록 함으로써 질의응답(QA), 자연어 추론(NLI) 등,<br/>
문장 관계를 이해해야만 하는 복잡한 태스크에서 좋은 성능을 나타내는 역할을 하였습니다.

- **Fine-tuning**

BERT 역시 모델의 구조는 그대로 유지한 채 데이터를 입력하는 형태만 바꾸어서 Fine-tuning을 실시합니다.

<img width="700" alt="nsp_2" src="http://jalammar.github.io/images/bert-tasks.png">

(a)는 “Sentence” 쌍을 분류하는 태스크입니다. `[SEP]`으로 나눠진 “Sentence” 쌍을 입력받아 `[CLS]`가 출력하는 클래스를 반환합니다.

(b)는 감성분석 등 하나의 문장을 입력하여 `[CLS]`로 해당 문장을 분류하는 태스크입니다.

(c)는 질의 응답 태스크입니다. 질문과 본문에 해당하는 단락을 `[SEP]` 토큰으로 나누어 입력하면 질문에 대한 답을 출력하도록 합니다.

(d)는 품사 태깅(POS tagging)이나 개체명 인식(Named Entity Recognition, NER) 등의 태스크입니다. 입력받은 각 토큰마다 답을 출력합니다.

- **결과 & 결론**

BERT는 간단한 사전 학습 아이디어로 많은 태스크에서 SOTA를 달성하였습니다.<br/>단순한 아이디어를 통해 엄청난 성능을 달성하였기에 당시 많은 충격을 주었습니다. 이후로도 BERT를 개선하기 위한 연구가 많이 진행되었습니다.

특히 MLM을 통해 BERT가 좋은 성능을 달성한 뒤로 텍스트에 노이즈를 준 후에 이를 다시 맞추는(Denoising) 방법에 대해 많은 연구가 진행되었습니다.

## 3. Post BERT (최근 NLP 연구 방향에 대해서)

### 1) 더 큰 모델 만들기

<img width="700" alt="getting_bigger" src="https://www.microsoft.com/en-us/research/uploads/prod/2020/02/TurningNGL_Model__1400x788.png">

GPT(2018.6)와 BERT(2018.10) 이후로도 수많은 모델이 발표되어 왔습니다.<br/>
두 모델 이후로 발표되고 있는 모델의 주요 경향성 중 하나는 **모델 크기 키우기** 입니다.<br/>
위 그림에서 볼 수 있듯  GPT 와 BERT이후 발표되는 모델의 파라미터 수는 기하급수적으로 증가하고 있습니다.

특히 2020년 6월에 발표된 GPT-3의 파라미터 개수는 약 1750억 개로 위 그림에 나와있는 T-NLG보다도 10배나 많습니다.<br/>
많은 모델이 크기를 키울수록 더 좋은 성능을 보여주고 있기 때문에 각 기업에서는 모델 크기를 계속해서 키우고 있는 상황입니다.<br/>

하지만 크기가 커지면 사전 학습에 따른 비용이 많이 들고 그만큼 많은 학습 데이터를 확보해야 합니다.<br/>
이런 제약사항 때문에 초대형 모델은 학계보다는 구글, 페이스북 등과 같은 대기업에서만 이루어지고 있습니다.<br/>
큰 모델에 비해 그렇지 못한 모델이 좋은 성능을 보장하지 못하고 있기 때문에, 학계에서는 상대적으로 뒤처질 수밖에 없게됩니다.<br/>
이런 사태가 계속되면서 크기만 커지는 모델에 대한 우려의 시각도 있습니다.

<img width="400" alt="getting_bigger_gpt3" src="https://miro.medium.com/max/1164/1*C-KNWQC_wXh-Q2wc6VPK1g.png">

### 2) 더 좋은 학습 방법 적용하기

여전히 더 좋은 학습 방법을 연구하고자 하는 움직임도 계속되고 있습니다.<br/>
특히 기존 GPT나 BERT의 단점을 보완하는 방향으로 많은 연구가 진행되고 있습니다.<br/>

트랜스포머의 디코더 블록만을 사용한 GPT는 상대적으로 자연어 생성과 관련된 태스크에,<br/>
인코더 블록만을 사용한 BERT는 자연어 이해와 관련된 태스크에 특화되어 있습니다.<br/>
GPT와 같이 순차적으로 자연어를 생성하는 모델에는 AR(Auto-Regressive)한 방법이 적용되었다고 하고,<br/>
BERT와 같이 노이즈를 맞추어가는 방식으로 자연어를 이해하는 모델에는 AE(Auto-Encoder)한 방법이 적용되었다고 합니다.

**두 모델이 사용했던 방법을 결합(AE+AR)**한 모델로 XLNet이나 BART가 있습니다.<br/>
두 모델 모두 자연어 이해와 생성 모두에서 좋은 성능을 보이며 특히 BART는 요약 태스크에서 좋은 성능을 보입니다.<br/>

**다른 방향의 개선으로는 BERT의 Noising 방법을 어렵게** 만든 모델들이 있습니다.<br/>
대표적인 모델로 Masking 방법에 변화를 주는 SpanBERT, RoBERTa와 같은 모델이 있습니다.

아래는 BART 모델을 나타내는 그림입니다.

<img width="500" alt="bart" src="https://miro.medium.com/max/1400/0*MeyyeTYxwtSZJPiL">   

위 그림처럼 AE 와 AR이 모두 적용되었습니다.<br/>
게다가 BART는 Masking 뿐만 아니라 Permutation, Infilling 등 다양한 Noising 방법이 적용되었다는 특징을 가지고 있습니다. 

<img width="500" alt="bart_noising" src="https://www.weak-learner.com/assets/img/blog/personal/bart_transformations.png">

### 3) 보다 가벼운 모델 만들기

GPT와 BERT 기본 모델이라도 크기가 꽤 크다 보니 사이즈를 줄이되 성능은 보전하는 경량화로도 많은 연구가 진행되고 있습니다.<br/>
DistillBERT, ALBERT(A Light BERT) 나 ELECTRA가 이런 방향으로 연구된 대표적인 모델이라고 할 수 있습니다.

세 모델 모두 각자만의 방법을 이용해서 BERT의 크기(=파라미터 수)를 많이 줄이고 성능은 어느정도 보존함으로써 모델 효율성을 높였습니다.

아래는 ELECTRA의 모델 구조입니다.<br/>
다음 시간에 배울 GAN에 등장하는 방법론을 적용하여 BERT보다 적은 리소스를 활용하여 더 높은 성능을 기록하였습니다.

<img width="500" alt="bart_noising" src="https://1.bp.blogspot.com/-sHybc03nJRo/XmfLongdVYI/AAAAAAAAFbI/a0t5w_zOZ-UtxYaoQlVkmTRsyFJyFddtQCLcBGAsYHQ/s1600/image1.png">

### 여러 방면에서의 다양한 시도

- **다양한 태스크를 수행할 수 있는 모델 (Meta Learning)**

T5나 GPT-3와 같은 모델은 하나의 모델로 다양한 태스크를 수행할 수 있는 모델입니다.<br/>
특히 GPT-3는 Few-shot learning 방법론을 적용한 모델로 적당한 길이의 제시문만 주어주면 **Fine-tuning 없이도** 엄청나게 좋은 성능을 보여줍니다.

N-shot learning 에 대해서는 아래 자료를 참고하시면 되겠습니다.

1. ***파인튜닝(finetuning)*** : 다운스트림 태스크에 해당하는 데이터 전체를 사용합니다. 모델 전체를 다운스트림 데이터에 맞게 업데이트합니다.

2. ***제로샷러닝(zero-shot learning)*** : 다운스트림 태스크 데이터를 전혀 사용하지 않습니다. 모델이 바로 다운스트림 태스크를 수행합니다.

3. ***원샷러닝(one-shot learning)*** : 다운스트림 태스크 데이터를 한 건만 사용합니다. 모델 전체를 1건의 데이터에 맞게 업데이트합니다. 업테이트 없이 수행하는 원샷러닝도 있습니다. 모델이 1건의 데이터가 어떻게 수행되는지 참고한 뒤 바로 다운스트림 태스크를 수행합니다.

4. ***퓨샷러닝(few-shot learning)*** : 다운스트림 태스크 데이터를 몇 건만 사용합니다. 모델 전체를 몇 건의 데이터에 맞게 업데이트합니다. 업데이트 없이 수행하는 퓨삿러닝도 있습니다. 모델이 몇 건의 데이터가 어떻게 수행되는지 참고한 뒤 바로 다운스트림 태스크를 수행합니다.

- **다국어(Multilingual) 모델**

**다국어 모델** 역시 열심히 연구되고 있는 분야입니다.<br/>
일반적인 언어 모델의 경우 단일 말뭉치로만 사전 학습을 진행하는 경우가 많습니다.<br/>
그렇기 때문에 사전 학습된 언어 외에 다른 언어를 사용하면 성능이 급격히 저하되는 경우가 많습니다.

예를 들어, 영어로 학습된 GPT-3 모델에 한국어를 집어넣으면 거의 이해하지 못하게 됩니다.<br/> 
이러한 문제를 뛰어넘기 위해 다양한 언어를 넘나들며 사용할 수 있는 모델이 연구되고 있는데 이를 다국어 모델 이라고 합니다.

대표적인 다국어 모델로는 mBART(multi-lingual BART), mT5(multi-lingual T5) 등이 있습니다.

- **자연어를 넘어(1) : 컴퓨터 비전(Computer Vision, CV)에서의 트랜스포머**

트랜스포머는 원래 자연어처리 중 번역 태스크에 적용하기 위해서 나온 모델이었습니다.<br/>
하지만 최근에는 컴퓨터 비전 태스크인 이미지 처리에서도 트랜스포머를 적용하고자 하는 움직임이 나타나고 있습니다.<br/>

[ViT(Vision in Transformer)](https://arxiv.org/abs/2010.11929) 논문에서는 컴퓨터 비전 분야에서 SOTA인 CNN 계열 모델보다 트랜스포머가 더 좋은 성능을 나타냈다고 말하고 있습니다.<br/>
(CNN 모델이 무엇인지 당장은 몰라도 좋습니다. Sprint 3에서 자세히 배우게 될 것입니다.)

아직 컴퓨터 비전 분야에서는 CNN에 비해 개발 속도가 더딘 편이지만,<br/>
트랜스포머를 사용하여 비전과 자연어 모두 정복하기 위한 다양한 시도가 지금도 진행되고 있습니다.

- **자연어를 넘어(2) : 멀티 모달(Multi-modal) 모델**

지난 1월에는 GPT를 발표했던 OpenAI에서 DALL-E 와 CLIP 이라는 재미있는 모델을 발표했습니다.<br/>
이 모델은 텍스트(문장)를 입력받아 상응하는 이미지를 생성합니다.

아래는 DALL-E 가 _"an armchair in the shape of an avocado"_ 라는 문장을 입력받은 뒤 출력한 이미지입니다.



<img width="700" alt="dall-e" src="https://user-images.githubusercontent.com/45377884/113083201-b9425500-9216-11eb-989a-3e5f28a794e5.png">

이렇게 자연어를 넘어 다양한 매체로 기계와 소통하는 태스크를 **멀티모달(Multi-Modal)**이라고 합니다.<br/>
트랜스포머가 자연어처리 뿐만 아니라 컴퓨터 비전에 대해서도 좋은 성능을 보이기 때문에 트랜스포머를 활용한 멀티모달 연구도 활발하게 진행되고 있습니다.

## Review

학습 목표에 대해 다시 생각해봅시다.

- Attention의 장점에 대해서 생각하고 설명해봅니다.

    - RNN 모델의 단점 2가지
    - 장기 의존성(Long-term dependency)
    - Attention의 장점

- Transformer의 장점과 구조에 대해서 생각하고 설명해봅니다.
    - "Attention is All You Need" (왜 논문 제목을 이렇게 지었을지에 대해서 생각해봅시다)
    - Positional Encoding
    - Self-Attention
    - Masked Self-Attention
    - Encoder-Decoder Attention


- GPT & BERT
    - 사전 학습 언어 모델(Pretrained Language Model), 전이 학습(Transfer Learning)
        - 사전 학습(Pre-training)
        - Fine-tuning
    - GPT의 구조
    - BERT의 구조
        - MLM(Masked Langauge Model)
        - NSP(Next Sentence Prediction)

        

## 참고 자료

- 트랜스포머에 대해 조금 더 자세하게 알고 싶다면
    - [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
    - [번역](https://nlpinkorean.github.io/illustrated-transformer/)
    - [Paper](https://arxiv.org/pdf/1706.03762.pdf) (Attention is All You Need)

- GPT에 대해 더 자세하게 알고 싶다면
    - [The Illustrated GPT-2](http://jalammar.github.io/illustrated-gpt2/) (Visualizing Transformer Language Models)
    - [Paper](https://www.cs.ubc.ca/~amuham01/LING530/papers/radford2018improving.pdf) (Improving Language Understanding by Generative Pre-Training)

- BERT에 대해 더 자세하게 알고 싶다면
    - [The Illustrated BERT, ELMo, and co.](http://jalammar.github.io/illustrated-bert/) (How NLP Cracked Transfer Learning)
    - [번역](https://nlpinkorean.github.io/illustrated-bert/)
    - [Paper](https://arxiv.org/pdf/1810.04805.pdf) (Pre-training of Deep Bidirectional Transformers for
Language Understanding)